# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
city_weather = pd.read_csv('../WeatherPy/city_weather.csv')

In [4]:
city_weather

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,joensuu,62.60,29.76,37.00,88,75,6.93,FI,1602806136
1,1,ushuaia,-54.80,-68.30,39.20,86,75,28.86,AR,1602806137
2,2,yellowknife,62.46,-114.35,27.00,63,90,21.92,CA,1602806139
3,3,aykhal,66.00,111.50,15.80,85,90,6.71,RU,1602806140
4,4,albany,42.60,-73.97,68.00,75,8,3.00,US,1602805843
...,...,...,...,...,...,...,...,...,...,...
455,455,port blair,11.67,92.75,80.65,83,100,10.76,IN,1602806690
456,456,kortkeros,61.81,51.58,30.20,100,90,6.71,RU,1602806692
457,457,tarbagatay,51.48,107.36,26.60,92,90,3.89,RU,1602806693
458,458,ucluelet,48.93,-125.55,54.00,81,90,8.05,CA,1602806598


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
# Plot heatmap
fig = gmaps.figure()

# Create heat layer
locations = city_weather[["Lat","Lng"]]
weight = city_weather["Humidity"]
#may have to add .astype(float)
heat_layer = gmaps.heatmap_layer(locations, weights=weight)
#may add other heatmap settings such as dissipating=False, max_intensity=10, point_radius=1

# Add layer
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# Drop null - this is erroring out - perhaps as there are no null values??
# print(len(city_weather))
# city_weather.dropna(inplace=true)
# print(len(city_weahter))
# Drop based on weather criteria
filter_criteria = ((city_weather["Max Temp"]>75) &
            (city_weather["Max Temp"]<=89) &
            (city_weather["Humidity"]<85) &
            (city_weather["Wind Speed"]<13) &
            (city_weather["Cloudiness"]<=40))

vac_cities = city_weather[filter_criteria]
vac_cities
#~before filter would reverse this and say keep everything except essentially.

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
26,26,brandon,27.94,-82.29,84.00,78,1,2.46,US,1602806167
54,54,san quintin,30.48,-115.95,80.74,39,0,9.42,MX,1602806200
62,62,pacific grove,36.62,-121.92,87.80,26,1,3.36,US,1602805914
69,69,faanui,-16.48,-151.75,79.90,76,38,10.02,PF,1602806218
74,74,salalah,17.02,54.09,77.00,73,40,5.82,OM,1602806224
81,81,quixada,-4.97,-39.02,76.30,81,34,9.53,BR,1602806232
84,84,bilma,18.69,12.92,81.95,18,0,0.92,NE,1602806235
93,93,mitsamiouli,-11.38,43.28,77.32,75,37,2.19,KM,1602806246
96,96,teseney,15.11,36.66,82.36,36,0,5.91,ER,1602806249
118,118,tindwari,25.62,80.53,79.59,68,0,5.35,IN,1602806275


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = vac_cities.loc[:,["City", "Country", "Lat", "Lng"]]
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
26,brandon,US,27.94,-82.29,
54,san quintin,MX,30.48,-115.95,
62,pacific grove,US,36.62,-121.92,
69,faanui,PF,-16.48,-151.75,
74,salalah,OM,17.02,54.09,
81,quixada,BR,-4.97,-39.02,
84,bilma,NE,18.69,12.92,
93,mitsamiouli,KM,-11.38,43.28,
96,teseney,ER,15.11,36.66,
118,tindwari,IN,25.62,80.53,


In [8]:
# get lat,lng from DataFrame
for index, row in hotel_df.iterrows():
    try:      
        # API base URL and parameters
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
        # add params dictionary to update iterations
        params = {
            "type":"lodging",
            "keyword":"hotel",
            "radius":5000,
            "key":g_key,
            }
    
        city = row["City"]
        lat = row["Lat"]
        lon = row["Lng"]
    
    
        params['location'] = f"{lat},{lon}"
        
        response = requests.get(base_url,  params=params).json()
        results = response['results']
        print(f"A hotel in {city} was found.")  
        hotel_df.loc[index, "Hotel Name"] = response['results'][0]["name"]
            
    except:
        print("Hotel not found ... skipping.")

A hotel in brandon was found.
A hotel in san quintin was found.
A hotel in pacific grove was found.
A hotel in faanui was found.
A hotel in salalah was found.
A hotel in quixada was found.
A hotel in bilma was found.
Hotel not found ... skipping.
A hotel in mitsamiouli was found.
A hotel in teseney was found.
A hotel in tindwari was found.
Hotel not found ... skipping.
A hotel in saint george was found.
A hotel in lompoc was found.
A hotel in port macquarie was found.
A hotel in atar was found.
A hotel in kieta was found.
Hotel not found ... skipping.
A hotel in jacareacanga was found.
A hotel in tremedal was found.
Hotel not found ... skipping.
A hotel in san cristobal was found.
A hotel in hervey bay was found.
A hotel in naze was found.
A hotel in buala was found.
A hotel in tecoanapa was found.
A hotel in general roca was found.
A hotel in kupang was found.
A hotel in araguacu was found.
A hotel in cayenne was found.
A hotel in cockburn town was found.
A hotel in ballina was found.

In [9]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
26,brandon,US,27.94,-82.29,Sheraton Tampa Brandon Hotel
54,san quintin,MX,30.48,-115.95,Old Mill Hotel
62,pacific grove,US,36.62,-121.92,Monterey Plaza Hotel & Spa
69,faanui,PF,-16.48,-151.75,InterContinental Bora Bora Resort Thalasso Spa
74,salalah,OM,17.02,54.09,Al Jabal
81,quixada,BR,-4.97,-39.02,Hotel Vale das Pedras
84,bilma,NE,18.69,12.92,
93,mitsamiouli,KM,-11.38,43.28,Al Camar Lodge
96,teseney,ER,15.11,36.66,Ferait Hotel
118,tindwari,IN,25.62,80.53,


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map

# Create a marker_layer using the poverty list to fill the info box
fig = gmaps.figure()
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(heat_layer)
    
fig.add_layer(markers)
fig


Figure(layout=FigureLayout(height='420px'))